In [3]:
def some_func(*login_names):
    return {"login": l for l in login_names}

some_func("twitch_user1", "twitch_user2", "acb")

{'login': 'acb'}

In [ ]:
from multidict import MultiDict

def get_logins(*login_names):
    # Jeder login_name soll ein eigener 'login'-Key sein
    md = MultiDict()
    for name in login_names:
        md.add('login', name)
    return md

x = get_logins("twitch_user1", "twitch_user2", "acb")


'twitch_user1'

In [ ]:
a = {"key": "value", "key2": "value2"}

b = {"other_key": "other_value", "other_key2": "other_value2"}

a.

a

{'key': 'value',
 'key2': 'value2',
 'other_key': 'other_value',
 'other_key2': 'other_value2'}

In [7]:
def func(**kwargs):
    d = {"key": "value", "kwargs": kwargs}
    return d

a = func(key2="value2", key3="value3")
a

{'key': 'value', 'kwargs': {'key2': 'value2', 'key3': 'value3'}}